In [5]:
from ts_config import GenerationConfig
import xarray
from distributed import Client
from dask.distributed import wait, TimeoutError
from uuid import uuid4
from shutil import rmtree
from pathlib import Path

client = Client('localhost:8786')
client

<Client: 'tcp://146.6.192.149:8786' processes=10 threads=10, memory=465.66 GiB>

In [2]:
config = GenerationConfig(["/local1/testing_timeseries/history_files/v21.LR.BSMYLEsmbb.1970-05.001/archive.001/"], "/local1/testing_timeseries/output/")
interm_sizes = config.fit_interm_timeseries_to_memory()
batches = config.get_timeseries_batches(interm_sizes)

Sampling dataset metadata from each case group to estimate total size in memory (this may take some time)...


In [14]:
%%time
for output_dir, group, batch_paths in batches:
    output_dir.mkdir(parents=True, exist_ok=True)
    history_zarr_path = f"{output_dir}/tmp_hs_store.zarr"

    history_concat = xarray.open_mfdataset(batch_paths, parallel=True)
    history_concat.to_zarr(history_zarr_path, mode="w", consolidated=True)

    dt = history_concat.time.values[1] - history_concat.time.values[0]
    if dt.days == 0:
        time_str_format = "%Y-%m-%d-%H"
    elif 30 > dt.days > 0:
        time_str_format = "%Y-%m-%d"
    else:
        time_str_format = "%Y-%m"
    
    time_start = history_concat.time.values[0].strftime(time_str_format)
    time_end = history_concat.time.values[-1].strftime(time_str_format)
    
    attribute_variables = []
    for variable in list(history_concat.variables):
        if "cell_methods" not in history_concat[variable].attrs:
            attribute_variables.append(variable)
    
    def export_dataset(config_tuple):
        zarr_path, variables, output_path, uid = config_tuple
        xarray.open_zarr(zarr_path)[variables].to_netcdf(output_path, mode="w")
        return uid
    
    config_tuples = [(f"/local1/testing_timeseries/output/full_concat.zarr",
                      [variable],
                      f"/local1/testing_timeseries/output/{group}.{variable}.nc",
                      uuid4())
                     for variable in list(history_concat.variables) if variable not in attribute_variables]
    
    futures = client.map(export_dataset, config_tuples)
    
    for task in futures:
        wait(task)
        task.release()
    client.amm.stop()
    scatted_attrs = client.scatter(attrs_ds, broadcast=True)
    
    def add_descriptive_variables(path_ds_tuple):
        ds, path = path_ds_tuple
        ds.to_netcdf(path, mode="a")
    
    path_tuples = [(scatted_attrs, f"/local1/testing_timeseries/output/{variable}.nc") for variable in list(history_concat.variables) if variable not in attribute_variables]
    futures = client.map(add_descriptive_variables, path_tuples)
    
    for task in futures:
        wait(task)
        task.release()
    scatted_attrs.release()
    
    rmtree(history_zarr_path)
    client.amm.start()

ValueError: Codec does not support buffers of > 2147483647 bytes